In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
import numpy as np
import keras
from keras.models import *
from keras.layers import *
from tensorflow.python.keras import losses
from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint
import cv2
import glob
import matplotlib.pyplot as plt
import itertools
from util import losses, custom_data_generator, metrics 
from keras.utils import multi_gpu_model
from models import model_loader
import datetime
from processing import abs_sobel_thresh
from processing import mag_threshold
from processing import *
from util import custom_data_generator as data_util
from models.common import lanenet_wavelet
# import tensorflow as tf
# from keras.backend.tensorflow_backend import set_session
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8
# set_session(tf.Session(config=config))


batch_size =30
is_train =  False
# model_name = 'lanenet'
image_width,image_height = 256,256
channeles = 3
# checkpoint_name = "checkpoint_lanenet"
one_hot_label= False
data_aug = False

 





Using TensorFlow backend.
/home/beemap/miniconda3/envs/cesar3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/beemap/miniconda3/envs/cesar3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/beemap/miniconda3/envs/cesar3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/beemap/mini

In [2]:
path = '/home/beemap/Documents/cesar-workspace/Lane_Data/Mix_dataset_full'
val_inputs_path = path +"/train/"
val_masks_path = path +"/train_labels/"


val_samples = glob.glob(val_inputs_path + "*")

print("Validation samples = %s\n\n"%(len(val_samples)))


Validation samples = 8927




In [2]:
bigImg = './full_img.png'
bigLane = './BigLane.png'
BigImg = cv2.imread(bigImg, -1)
BigLane = cv2.imread(bigLane, -1)

In [3]:
BigImg.shape, BigLane.shape

((4608, 5120, 3), (4608, 5120, 3))

In [4]:
checkpoint_name = "GAN_v4_label_normal_lanenet2"
model_name = 'lanenet2'
checkpoint_dir = "./checkpoints/%s/"%(checkpoint_name)


print("Loading Model .... ",checkpoint_dir)
json_file = open(checkpoint_dir+model_name+".json", 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("./checkpoints/%s/%s_weights_90.h5"%(checkpoint_name,model_name))


Loading Model ....  ./checkpoints/GAN_v4_label_normal_lanenet2/


In [26]:
new_dir = './predictions_lanes_lines_v5/'
if not os.path.isdir(new_dir):
    os.makedirs(new_dir)
 

In [17]:
def get_hough_lines(img, rho=1, theta=np.pi/180, threshold=20, min_line_len=20, max_line_gap=300):
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
    return lines
def draw_lines(img, lines, color=[255, 255, 255], thickness=1):
    
    
    if len(img.shape) == 2:
        img = np.stack([img] * 3, axis=-1)
    if (type(lines) != np.ndarray):
        return img
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

    return img

def grayscale(img):
    
    if len(img.shape) > 2:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    return img

def hough_lines(img, rho=1, min_line_len=13,threshold=70, max_line_gap=300):
#     img = grayscale(img)
#     print(img.shape)
    edges = cv2.Canny(img, 80, 200)
    hough = get_hough_lines(edges, rho=rho, min_line_len=min_line_len, threshold=threshold, max_line_gap=max_line_gap)
    canvas = np.zeros_like(img)
    return draw_lines(canvas, hough)

In [27]:
from keras.callbacks import TensorBoard,Callback
from PIL import Image
import shutil
import matplotlib.pyplot as plt

n = 29
m = 20

i = 0
for image in val_samples[:]:
    full_img = cv2.imread(image, 1)#[:, 256:, :3]
    img_ = full_img[:, :, :3]
#     segmented = full_img[:, 512:, :]
    h, w,_ = img_.shape
#     lane = 
    tmp = img_[:]
    r_img = img_

    img_ = np.float32(img_)/255.0
    name = os.path.basename(image)
    maskPath = val_masks_path + name#[3:]
    lane = cv2.imread(maskPath, -1)/255.0
    lane_gray = cv2.imread(maskPath, 0)
    hough = hough_lines(lane_gray, rho=1, min_line_len=10,threshold=40, max_line_gap=300)/255.0
    lane += hough
#     lane/=255.0
    img_ = np.concatenate((img_, lane), axis=1)
#     print(img_.shape)
    if img_.shape[0] ==(512):
        image_list = []
        for n in range(0, 512, 256):
            for m in range(0, 512, 256):
                image_list.append(img_[n:n+256, m:m+256, :])
    else:
        image_list = [img_]
    seg_list = []
    name = os.path.basename(image)      
    for i, img in enumerate(image_list):
#         input_image_gray = data_util.get_image(image,do_aug=[],gray=True, change=False)
        
        img0 = img[:, :256, :]
        img1 = img[:, 256:, :]
#         print(img0.shape)
        input_image_gray = cv2.imread(image, 0)#cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
        w1, w2, w3, w4 = lanenet_wavelet(input_image_gray)
        w1 = np.expand_dims(w1, axis=0)
        w2 = np.expand_dims(w2, axis=0)
        w3 = np.expand_dims(w3, axis=0)
        w4 = np.expand_dims(w4, axis=0)
#         print(img.shape)
        mask = model.predict([np.expand_dims(img0, axis=0), np.expand_dims(img1, axis=0), w1, w2, w3, w4], batch_size=None, verbose=0, steps=None)

#         mask = model.predict([np.expand_dims(img, axis=0), w1, w2, w3, w4], batch_size=None, verbose=0, steps=None)
        #print(mask.shape)
        mask = np.round(mask[0, :, :, 0]).astype(int)
        #print(mask.shape)
        seg = np.zeros((256, 256, 3))

        seg[:, :, 0] += ((mask[:, :] == 1) * (255)).astype('uint8')
        seg[:, :, 1] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg[:, :, 2] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg_list.append(seg)
    final_seg = np.zeros((h, w, 3))
    for i,seg in enumerate(seg_list):
        if i == 0:
            final_seg[:256, :256, :] = seg
        elif i == 1:
            final_seg[:256, 256:, :] = seg
        elif i == 3:
            final_seg[256:, 256:, :] = seg
        else:
            final_seg[256:, :256, :] = seg
            
            
#         pred_list.append(seg)
    dest = new_dir + name#[:-4] + '_' + '.png'
#     index = np.all((segmented == np.array((0, 0, 0)).reshape(1, 1, 3)), axis=2)
#     final_seg[index] = np.array((0, 0, 0))
#         r = 0 if i < 2 else 1
#         c = 0 if i % 2 == 0 else 1
        
#         bigImg[r*256 : (r+1)*256, c*256 : (c+1)*256, :] = img*255
#         bigMask[r*256 : (r+1)*256, c*256 : (c+1)*256, :] = seg
#         filename = new_dir + name
#     seg = cv2.resize(seg, (512, 512))
#         cv2.imwrite(filename, np.concatenate((bigImg, bigMask), axis=1))
    merge = np.concatenate((img0*255, img1*255, final_seg), axis=1)
#         pred_list.append(merge)
    cv2.imwrite(dest, merge)
    

In [14]:
BigLane_gray = cv2.imread(bigLane, 0)

In [18]:
from keras.callbacks import TensorBoard,Callback
from PIL import Image
import shutil
import matplotlib.pyplot as plt

BigLine = np.zeros_like((BigImg))

for r in range(0, BigImg.shape[0], 256):
    for c in range(0, BigImg.shape[1], 256):
        img = BigImg[r:r+256, c:c+256, :3]
#         h, w, _ = img.shape
        tmp = img[:]
        lane = BigLane[r:r+256, c:c+256, :3]
        img = np.float32(img)/255.0
        lane = np.float32(lane)/255.0
        
#         input_image_gray = data_util.get_image(image,do_aug=[],gray=True, change=False)
        input_image_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        w1, w2, w3, w4 = lanenet_wavelet(input_image_gray)
        w1 = np.expand_dims(w1, axis=0)
        w2 = np.expand_dims(w2, axis=0)
        w3 = np.expand_dims(w3, axis=0)
        w4 = np.expand_dims(w4, axis=0)
        lane_gray = BigLane_gray[r:r+256, c:c+256]
#         lane_gray = cv2.cvtColor(lane, cv2.COLOR_BGR2GRAY)
        
        hough = hough_lines(lane_gray, rho=1, min_line_len=10,threshold=40, max_line_gap=300)/255.0
        mask = model.predict([np.expand_dims(img, axis=0), np.expand_dims(lane, axis=0), w1, w2, w3, w4], batch_size=None, verbose=0, steps=None)

        mask = np.round(mask[0, :, :, 0]).astype(int)
        seg = np.zeros((256, 256, 3))

        seg[:, :, 0] += ((mask[:, :] == 1) * (255)).astype('uint8')
        seg[:, :, 1] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        seg[:, :, 2] += ((mask[:, :] == 1) * ( 255)).astype('uint8')
        
        BigLine[r:r+256, c:c+256, :] = seg
name = 'BigLine2.png' 
new_dir = './'
dest = new_dir + name
cv2.imwrite(dest, BigLine)


True